In [ ]:
# 📌 Import Required Libraries
import os
import streamlit as st
from dotenv import load_dotenv
from typing_extensions import TypedDict
from typing import Annotated
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# ✅ Load API Keys (Set up in .env file)
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# ✅ Initialize LLM (GPT-4 for Interview Questions & Analysis)
llm = ChatOpenAI(model="gpt-4", temperature=0)

# ✅ Define AI State (Tracking Interview Progress)
class InterviewState(TypedDict):
    job_description: str
    interview_questions: list
    current_question: str
    answer: str
    feedback: str
    score: int
    final_feedback: str

# ✅ Step 1: Generate Questions Based on Job Description
generate_questions_prompt = PromptTemplate(
    input_variables=["job_description"],
    template="Based on this job description, generate 5 interview questions:\n{job_description}"
)
generate_questions_chain = LLMChain(llm=llm, prompt=generate_questions_prompt)

def generate_questions(state: InterviewState):
    """Creates interview questions based on job description"""
    questions = generate_questions_chain.run(state["job_description"]).split("\n")
    return {"interview_questions": questions, "current_question": questions[0]}

# ✅ Step 2: Analyze Candidate Answer
analyze_answer_prompt = PromptTemplate(
    input_variables=["current_question", "answer"],
    template="Evaluate this answer based on clarity, correctness, and depth.\nQuestion: {current_question}\nAnswer: {answer}\nProvide a score out of 10."
)
analyze_answer_chain = LLMChain(llm=llm, prompt=analyze_answer_prompt)

def analyze_answer(state: InterviewState):
    """Evaluates candidate's answer"""
    score = int(analyze_answer_chain.run(state["current_question"], state["answer"]).split()[0])
    return {"score": score}

# ✅ Step 3: Provide Feedback on Answer
feedback_prompt = PromptTemplate(
    input_variables=["answer", "score"],
    template="Provide constructive feedback on this answer based on its score ({score}/10).\nAnswer: {answer}"
)
feedback_chain = LLMChain(llm=llm, prompt=feedback_prompt)

def provide_feedback(state: InterviewState):
    """Gives feedback based on the answer score"""
    feedback = feedback_chain.run(state["answer"], state["score"])
    return {"feedback": feedback}

# ✅ Conditional Step: Retry Weak Answers or Proceed to Next Question
def should_retry(state: InterviewState):
    """Determines if the candidate should retry the answer"""
    if state["score"] < 6:
        return "retry_question"
    elif len(state["interview_questions"]) > 1:
        # Move to the next question if available
        state["interview_questions"].pop(0)
        state["current_question"] = state["interview_questions"][0]
        return "next_question"
    else:
        return "generate_final_feedback"

# ✅ Step 4: Generate Final Feedback
final_feedback_prompt = PromptTemplate(
    input_variables=["score"],
    template="Based on the interview performance with an average score of {score}/10, provide a final evaluation."
)
final_feedback_chain = LLMChain(llm=llm, prompt=final_feedback_prompt)

def generate_final_feedback(state: InterviewState):
    """Generates final performance feedback"""
    final_feedback = final_feedback_chain.run(state["score"])
    return {"final_feedback": final_feedback}

# ✅ Build LangGraph Workflow
workflow = StateGraph(InterviewState)

# ➤ Add Nodes
workflow.add_node("generate_questions", generate_questions)
workflow.add_node("analyze_answer", analyze_answer)
workflow.add_node("provide_feedback", provide_feedback)
workflow.add_node("generate_final_feedback", generate_final_feedback)

# ➤ Define Execution Flow with Conditional Routing
workflow.add_edge(START, "generate_questions")  # Start → Generate Questions
workflow.add_edge("generate_questions", "analyze_answer")  # Questions → Analyze Answer

# Conditional: If score < 6 → Retry, else → Next Question or Final Feedback
workflow.add_conditional_edges(
    "analyze_answer",
    should_retry,
    {"retry_question": "provide_feedback", "next_question": "analyze_answer", "generate_final_feedback": "generate_final_feedback"}
)

# Retry process before moving to the next question
workflow.add_edge("provide_feedback", "analyze_answer")

# End after generating final performance feedback
workflow.add_edge("generate_final_feedback", END)

# ✅ Compile the Graph
interview_graph = workflow.compile()

# ✅ Streamlit Web App
st.title("🎤 AI Mock Interview System")
st.markdown("**Enter a job description, and AI will generate questions & evaluate your answers!**")

# User input for job description
job_description = st.text_area("Enter Job Description:", "Looking for a Python Developer with experience in Flask, SQL, and REST APIs.")

if st.button("Start Interview"):
    # Initialize state
    state = {"job_description": job_description}
    state = interview_graph.invoke(state)

    st.session_state["state"] = state

if "state" in st.session_state:
    st.markdown(f"**Question:** {st.session_state['state'].get('current_question', 'N/A')}")

    # Candidate answers
    answer = st.text_area("Your Answer:")
    if st.button("Submit Answer"):
        st.session_state["state"]["answer"] = answer
        st.session_state["state"] = interview_graph.invoke(st.session_state["state"])

        # Show feedback
        st.markdown(f"**Feedback:** {st.session_state['state'].get('feedback', 'N/A')}")
        st.markdown(f"**Score:** {st.session_state['state'].get('score', 0)}")

    # Show final performance evaluation
    if "final_feedback" in st.session_state["state"]:
        st.markdown(f"**Final Evaluation:** {st.session_state['state']['final_feedback']}")



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from typing_extensions import TypedDict
from typing import Annotated
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
llm = ChatGroq()

In [17]:
from IPython.display import Image, display

In [4]:
class InterviewState(TypedDict):
    job_description: str
    interview_questions: list
    current_question: str
    answer: str
    feedback: str
    score: int
    final_feedback: str

In [6]:
generate_questions_prompt = PromptTemplate(
    input_variables=["job_description"],
    template="Based on this job description, generate 5 interview questions:\n{job_description}"
)
generate_questions_chain = generate_questions_prompt|llm

def generate_questions(state: InterviewState):
    """Creates interview questions based on job description"""
    questions = generate_questions_chain.run(state["job_description"]).split("\n")
    return {"interview_questions": questions, "current_question": questions[0]}


In [8]:
analyze_answer_prompt = PromptTemplate(
    input_variables=["current_question", "answer"],
    template="Evaluate this answer based on clarity, correctness, and depth.\nQuestion: {current_question}\nAnswer: {answer}\nProvide a score out of 10."
)
analyze_answer_chain = analyze_answer_prompt|llm

def analyze_answer(state: InterviewState):
    """Evaluates candidate's answer"""
    score = int(analyze_answer_chain.run(state["current_question"], state["answer"]).split()[0])
    return {"score": score}


In [9]:
feedback_prompt = PromptTemplate(
    input_variables=["answer", "score"],
    template="Provide constructive feedback on this answer based on its score ({score}/10).\nAnswer: {answer}"
)
feedback_chain = feedback_prompt|llm

def provide_feedback(state: InterviewState):
    """Gives feedback based on the answer score"""
    feedback = feedback_chain.run(state["answer"], state["score"])
    return {"feedback": feedback}


In [10]:
def should_retry(state: InterviewState):
    """Determines if the candidate should retry the answer"""
    if state["score"] < 6:
        return "retry_question"
    elif len(state["interview_questions"]) > 1:
        # Move to the next question if available
        state["interview_questions"].pop(0)
        state["current_question"] = state["interview_questions"][0]
        return "next_question"
    else:
        return "generate_final_feedback"


In [11]:
final_feedback_prompt = PromptTemplate(
    input_variables=["score"],
    template="Based on the interview performance with an average score of {score}/10, provide a final evaluation."
)
final_feedback_chain = LLMChain(llm=llm, prompt=final_feedback_prompt)

def generate_final_feedback(state: InterviewState):
    """Generates final performance feedback"""
    final_feedback = final_feedback_chain.run(state["score"])
    return {"final_feedback": final_feedback}


In [12]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(InterviewState)

# ➤ Add Nodes
workflow.add_node("generate_questions", generate_questions)
workflow.add_node("analyze_answer", analyze_answer)
workflow.add_node("provide_feedback", provide_feedback)
workflow.add_node("generate_final_feedback", generate_final_feedback)

# ➤ Define Execution Flow with Conditional Routing
workflow.add_edge(START, "generate_questions")  
workflow.add_edge("generate_questions", "analyze_answer")  

workflow.add_conditional_edges(
    "analyze_answer",
    should_retry,
    {"retry_question": "provide_feedback", "next_question": "analyze_answer", "generate_final_feedback": "generate_final_feedback"}
)

workflow.add_edge("provide_feedback", "analyze_answer")
workflow.add_edge("generate_final_feedback", END)

# ✅ Compile the Graph
interview_graph = workflow.compile()


In [16]:
interview_graph.invoke({"job_description":"""AI Engineer will contributing to the development and deployment of NLP, LLM and Machine Learning. AI Engineer will play a crucial role in advancing our Generative AI capabilities and driving innovation across multiple projects. The person will collaborate closely with cross-functional teams to design, implement, and optimize algorithms that generate high-quality content. Experience working with various genAI toolkits like Langchain, LlamaIndex, Huggingface Transformers, Spacy, CoreNLP, OpenNLP etc. Proven experience with natural language processing, including textual data processing , training, and evaluating deep learning and large language models (Transformer based models, Bert, GPT, etc Languages: Python, AQL Platform: Azure, AWS Libraries: NLTK, scikit-learn. Hugging Face transformers, spaCy LLM - GPT3, GPT4 Primary Skills: GenAI, LLM, Cloud (AWS), Data Science, NLP"""})

AttributeError: 'RunnableSequence' object has no attribute 'run'